In this notebook, we firstly query the patent data from Google Patents Public Data using BigQuery. Afterward, we gather the list of patent assignees from the patent dataset, search for their financial data on S&P Capital IQ Compustat Database, and match the financial data with the patent data set. Lastly, we label the dataset to make it ready for the financial prediction with patent text task. 

The complete labeled dataset that is used for the analysis is stored in "data.zip" file. Downloading the zip data file, one can skip this data collection and labeling section and directly start with the analysis.

### Libraries

In [ ]:
# Download necessary libraries 
import pandas as pd
import numpy as np

import os, pickle
from pathlib import Path

from google.colab import drive
from google.cloud import bigquery

In [ ]:
# Mount Drive account on Colab
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
# Set BigQuery application credentials
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "path/to/credentials.json"

project_id = "your_bq_project_id"
bq_client = bigquery.Client(project=project_id)

### Dataset Query

In [ ]:
# Querying the patent data in two batches (2010-2013 and 2014-2019) due to its large size

query_1 = r'''
  #standardSQL
  SELECT 
    p.publication_number as publication_number, p.application_number as application_number, publication_date, p.filing_date as filing_date, 
    grant_date, priority_date, cpc.code, ipc.code as ipc_code, title_localized.text as title, 
    abstract_localized.text as abstract, assignee_harmonized.name as assignee_harmonized, assignee_harmonized.country_code as assignee_country, 
    inventor_harmonized.name as inventor_harmonized, inventor_harmonized.country_code as inventor_country
  FROM `patents-public-data.patents.publications` as p,
    UNNEST(cpc) as cpc,
    UNNEST(title_localized) as title_localized,
    UNNEST(abstract_localized) as abstract_localized,
    UNNEST(assignee_harmonized) as assignee_harmonized,
    UNNEST(inventor_harmonized) as inventor_harmonized,
    UNNEST(ipc) as ipc
  WHERE publication_date >= 20100101 AND publication_date < 20140101
    AND p.country_code = 'EP'
    AND cpc.code LIKE 'Y02%'
    AND title_localized.language = 'en'
    AND abstract_localized.language = 'en'
'''

In [ ]:
query_2 = r'''
  #standardSQL
  SELECT 
    p.publication_number as publication_number, p.application_number as application_number, publication_date, p.filing_date as filing_date, 
    grant_date, priority_date, cpc.code, ipc.code as ipc_code, title_localized.text as title, 
    abstract_localized.text as abstract, assignee_harmonized.name as assignee_harmonized, assignee_harmonized.country_code as assignee_country, 
    inventor_harmonized.name as inventor_harmonized, inventor_harmonized.country_code as inventor_country
  FROM `patents-public-data.patents.publications` as p,
    UNNEST(cpc) as cpc,
    UNNEST(title_localized) as title_localized,
    UNNEST(abstract_localized) as abstract_localized,
    UNNEST(assignee_harmonized) as assignee_harmonized,
    UNNEST(inventor_harmonized) as inventor_harmonized,
    UNNEST(ipc) as ipc
  WHERE publication_date >= 20140101 AND publication_date < 20200101
    AND p.country_code = 'EP'
    AND cpc.code LIKE 'Y02%'
    AND title_localized.language = 'en'
    AND abstract_localized.language = 'en'
'''

In [ ]:
df_1 = bq_client.query(query_1).to_dataframe()
df_2 = bq_client.query(query_2).to_dataframe()

In [ ]:
df = pd.concat([df_1, df_2], ignore_index=True)

In [ ]:
df.to_csv('patent_data.csv', index=False)

In [ ]:
!cp patent_data.csv "drive/My Drive/"

### Assignee list

The assignee list from the patents between 2010-2019 is gathered to have the full list of the companies. The assignee names will be matched with company S&P Capital IQ identifiers using SPCIQ Identifier Converter embedded in Capital IQ’s Excel Plugin tools. Then, the company financial data will be obtained from the Compustat Database using these identifiers. 

In [ ]:
df = pd.read_csv('/content/drive/My Drive/patent_data.csv')

In [ ]:
assignee = df[['assignee_harmonized']]

In [ ]:
assignee['assignee_harmonized'] = assignee['assignee_harmonized'].astype(str).str.strip()

In [ ]:
assignee = assignee.drop_duplicates(subset=['assignee_harmonized'], keep='first', ignore_index=True)

In [ ]:
assignee.to_excel('assignee.xlsx', index=False)

### Matching patents with the financials

Companies' financial data is obtained from the Compustat Database and saved in assignee_fin.csv. Now, we will merge the dataset of financials and patent dataset based on the company names.

In [ ]:
fin = pd.read_csv('/content/drive/My Drive/assignee_fin.csv')
df = pd.read_csv('/content/drive/My Drive/patent_data.csv')

In [ ]:
df = df.astype(str)

In [ ]:
cols=['application_number','publication_date', 'filing_date', 'grant_date', 'priority_date', 'code', 'ipc_code', 'title', 'abstract', 'assignee_harmonized', 'assignee_country', 'inventor_harmonized', 'inventor_country']

In [ ]:
# Grouping by publication number and assignees: getting unique values into a list - all columns need to be string

df = df.groupby(['publication_number', 'assignee_harmonized'],as_index=False)[cols].agg(lambda x: ', '.join(set(x))).reindex(columns=df.columns)


In [ ]:
fin.dropna(subset = ['identifier'], inplace=True)
fin.reset_index(drop=True)

In [ ]:
str_cols = list(fin.iloc[:,-80:84])    
fin[str_cols] = fin[str_cols].replace('NM', np.nan, regex=True)
fin[str_cols] = fin[str_cols].replace(',', '.', regex=True)

In [ ]:
fin = fin.rename({'assignee_harmonized': 'company'}, axis=1) 

In [ ]:
fin.head()

,assignee_fin,company,result_name,identifier,total_rev_2010,total_rev_2011,total_rev_2012,total_rev_2013,total_rev_2014,total_rev_2015,total_rev_2016,total_rev_2017,total_rev_2018,total_rev_2019,gross_profit_2010,gross_profit_2011,gross_profit_2012,gross_profit_2013,gross_profit_2014,gross_profit_2015,gross_profit_2016,gross_profit_2017,gross_profit_2018,gross_profit_2019,total_assets_2010,total_assets_2011,total_assets_2012,total_assets_2013,total_assets_2014,total_assets_2015,total_assets_2016,total_assets_2017,total_assets_2018,total_assets_2019,ebitda_2010,ebitda_2011,ebitda_2012,ebitda_2013,ebitda_2014,ebitda_2015,ebitda_2016,ebitda_2017,ebitda_2018,ebitda_2019,total_assets_growth_2010,total_assets_growth_2011,total_assets_growth_2012,total_assets_growth_2013,total_assets_growth_2014,total_assets_growth_2015,total_assets_growth_2016,total_assets_growth_2017,total_assets_growth_2018,total_assets_growth_2019,ebitda_growth_2010,ebitda_growth_2011,ebitda_growth_2012,ebitda_growth_2013,ebitda_growth_2014,ebitda_growth_2015,ebitda_growth_2016,ebitda_growth_2017,ebitda_growth_2018,ebitda_growth_2019,roa_2010,roa_2011,roa_2012,roa_2013,roa_2014,roa_2015,roa_2016,roa_2017,roa_2018,roa_2019,roe_2010,roe_2011,roe_2012,roe_2013,roe_2014,roe_2015,roe_2016,roe_2017,roe_2018,roe_2019
0,"Target Discovery, Inc.",TARGET DISCOVERY INC,"Target Discovery, Inc.",IQ1669583,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Basf Se,BASF SE,BASF SE,IQ382826,63873.00,73497.00,72129.00,73973.00,74326.00,70449.00,57550.00,61223.00,60220.00,59316.00,18433.00,19336.00,17686.00,18117.00,18262.00,18818.00,18008.00,19412.00,17064.00,15969.00,59393.00,61175.00,62726.00,64204.00,71359.00,70836.00,76496.00,78768.00,86556.00,86950.00,11762.00,11611.00,9191.00,9882.00,10027.00,10576.00,9725.00,10984.00,8976.00,7068.00,15.85,3.00,2.54,2.36,11.14,-0.73,7.99,2.97,9.89,0.46,52.52,-1.28,-20.84,7.52,1.47,5.48,-8.05,12.95,-18.28,-21.26,9.71,8.86,6.31,6.74,6.42,5.73,4.91,5.57,4.46,2.66,24.59,27.49,19.87,19.19,19.66,14.40,13.27,16.61,11.62,6.49
3,"Delphi Technologies, Inc.",DELPHI TECH INC,Delphi Technologies PLC,IQ433871117,NaN,NaN,NaN,NaN,3750.72,4057.75,4249.59,4038.25,4242.98,3886.09,NaN,NaN,NaN,NaN,717.92,790.92,782.47,816.14,784.31,584.56,NaN,NaN,NaN,NaN,NaN,2763.17,2746.22,3158.81,3400.15,3338.95,NaN,NaN,NaN,NaN,568.39,647.29,653.64,609.61,576.44,406.34,NaN,NaN,NaN,NaN,NaN,NaN,-3.40,30.84,2.64,-3.75,NaN,NaN,NaN,NaN,NaN,2.18,-1.85,6.09,-9.84,-30.91,NaN,NaN,NaN,NaN,NaN,NaN,10.78,10.14,7.55,4.12,NaN,NaN,NaN,NaN,NaN,NaN,21.24,45.12,113.43,7.39
4,Showa Denko K.K.,SHOWA DENKO KK,Showa Denko K.K.,IQ874164,7327.34,8545.47,6486.00,5854.80,6018.54,5942.22,5445.73,5770.02,7898.02,7432.49,1166.66,1348.97,974.94,753.11,749.10,933.07,1023.69,1313.20,2285.76,1929.32,8497.37,9417.31,8182.63,6807.59,6963.66,7204.33,7567.84,7576.62,8563.61,8825.82,832.78,980.15,662.34,463.93,433.31,589.83,655.64,859.60,1746.90,1300.44,-3.53,1.82,-0.86,5.64,2.44,-6.87,-0.83,9.87,4.98,0.06,80.03,8.13,-22.90,-11.06,-6.46,22.54,4.94,43.88,88.75,-27.73,2.57,3.17,1.87,1.69,1.29,2.15,2.81,4.97,10.71,7.02,5.25,6.78,3.61,3.11,0.52,-2.52,4.47,10.58,28.12,15.29
6,Toto Ltd.,TOTO LTD,Toto Ltd.,IQ873962,3338.55,3689.41,4120.65,3946.16,3901.55,4224.15,4437.83,4812.87,4526.82,4712.60,1157.53,1364.08,1509.98,1433.75,1492.16,1607.59,1690.22,1854.81,1713.15,1724.92,2993.06,3226.98,3432.36,3384.23,3358.31,4010.71,4190.70,4646.61,4326.81,4623.13,214.95,287.28,337.97,355.32,437.81,424.00,504.32,568.51,565.26,510.70,-2.67,0.25,-0.57,8.32,16.63,8.52,3.73,3.31,2.19,1.56,-6.30,24.27,9.98,15.50,44.82,-12.00,18.08,5.03,9.12,-14.12,1.07,2.31,3.10,3.72,6.67,4.71,5.48,5.57,5.87,4.40,0.85,3.25,5.52,8.83,19.26,9.71,12.86,11.76,11.67,9.63


In [ ]:
data = pd.merge(df, fin, left_on='assignee_harmonized', right_on="company") 
# 149123 total rows
# 143198 publication numbers

### Labeling the dataset for modeling

In [ ]:
# Preparing the date column
data['publication_date'] = data['publication_date'].astype(str).str.strip()
data['publication_date'] = pd.to_datetime(data['publication_date'])

In [ ]:
# Preparing the financial columns 
data[str_cols] = data[str_cols].astype(str)
data[str_cols] = data[str_cols].apply(pd.to_numeric, errors='coerce')

In [ ]:
str_cols = list(map(lambda st: str.replace(st, '_2010',''), str_cols)) 
str_cols = list(map(lambda st: str.replace(st, '_2011',''), str_cols)) 
str_cols = list(map(lambda st: str.replace(st, '_2012',''), str_cols)) 
str_cols = list(map(lambda st: str.replace(st, '_2013',''), str_cols)) 
str_cols = list(map(lambda st: str.replace(st, '_2014',''), str_cols)) 
str_cols = list(map(lambda st: str.replace(st, '_2015',''), str_cols)) 
str_cols = list(map(lambda st: str.replace(st, '_2016',''), str_cols)) 
str_cols = list(map(lambda st: str.replace(st, '_2017',''), str_cols)) 
str_cols = list(map(lambda st: str.replace(st, '_2018',''), str_cols)) 
str_cols = list(map(lambda st: str.replace(st, '_2019',''), str_cols)) 

In [ ]:
# Creating the label for the 1-year change 

for x in str_cols: 
    conditions = [ 
    (data['publication_date'].dt.year == 2010) & (data[x+'_2011'] == data[x+'_2010']) | (data['publication_date'].dt.year == 2011) & (data[x+'_2012'] == data[x+'_2011']) | (data['publication_date'].dt.year == 2012) & (data[x+'_2013'] == data[x+'_2012']) | (data['publication_date'].dt.year == 2013) & (data[x+'_2014'] == data[x+'_2013']) | (data['publication_date'].dt.year == 2014) & (data[x+'_2015'] == data[x+'_2014']) | (data['publication_date'].dt.year == 2015) & (data[x+'_2016'] == data[x+'_2015']) | (data['publication_date'].dt.year == 2016) & (data[x+'_2017'] == data[x+'_2016']) | (data['publication_date'].dt.year == 2017) & (data[x+'_2018'] == data[x+'_2017']) | (data['publication_date'].dt.year == 2018) & (data[x+'_2019'] == data[x+'_2018']),
    (data['publication_date'].dt.year == 2010) & (data[x+'_2011'] > data[x+'_2010']) | (data['publication_date'].dt.year == 2011) & (data[x+'_2012'] > data[x+'_2011']) | (data['publication_date'].dt.year == 2012) & (data[x+'_2013'] > data[x+'_2012']) | (data['publication_date'].dt.year == 2013) & (data[x+'_2014'] > data[x+'_2013']) | (data['publication_date'].dt.year == 2014) & (data[x+'_2015'] > data[x+'_2014']) | (data['publication_date'].dt.year == 2015) & (data[x+'_2016'] > data[x+'_2015']) | (data['publication_date'].dt.year == 2016) & (data[x+'_2017'] > data[x+'_2016']) | (data['publication_date'].dt.year == 2017) & (data[x+'_2018'] > data[x+'_2017']) | (data['publication_date'].dt.year == 2018) & (data[x+'_2019'] > data[x+'_2018']),
    (data['publication_date'].dt.year == 2010) & (data[x+'_2011'] < data[x+'_2010']) | (data['publication_date'].dt.year == 2011) & (data[x+'_2012'] < data[x+'_2011']) | (data['publication_date'].dt.year == 2012) & (data[x+'_2013'] < data[x+'_2012']) | (data['publication_date'].dt.year == 2013) & (data[x+'_2014'] < data[x+'_2013']) | (data['publication_date'].dt.year == 2014) & (data[x+'_2015'] < data[x+'_2014']) | (data['publication_date'].dt.year == 2015) & (data[x+'_2016'] < data[x+'_2015']) | (data['publication_date'].dt.year == 2016) & (data[x+'_2017'] < data[x+'_2016']) | (data['publication_date'].dt.year == 2017) & (data[x+'_2018'] < data[x+'_2017']) | (data['publication_date'].dt.year == 2018) & (data[x+'_2019'] < data[x+'_2018'])
    ]
    values = ['0', '1', '-1']
    data[x+'_label_1'] = np.select(conditions, values, default=None)

In [ ]:
# Creating the label for the 2-year change 

for x in str_cols: 
    conditions = [ 
    (data['publication_date'].dt.year == 2010) & (data[x+'_2012'] == data[x+'_2010']) | (data['publication_date'].dt.year == 2011) & (data[x+'_2013'] == data[x+'_2011']) | (data['publication_date'].dt.year == 2012) & (data[x+'_2014'] == data[x+'_2012']) | (data['publication_date'].dt.year == 2013) & (data[x+'_2015'] == data[x+'_2013']) | (data['publication_date'].dt.year == 2014) & (data[x+'_2016'] == data[x+'_2014']) | (data['publication_date'].dt.year == 2015) & (data[x+'_2017'] == data[x+'_2015']) | (data['publication_date'].dt.year == 2016) & (data[x+'_2018'] == data[x+'_2016']) | (data['publication_date'].dt.year == 2017) & (data[x+'_2019'] == data[x+'_2017']),
    (data['publication_date'].dt.year == 2010) & (data[x+'_2012'] > data[x+'_2010']) | (data['publication_date'].dt.year == 2011) & (data[x+'_2013'] > data[x+'_2011']) | (data['publication_date'].dt.year == 2012) & (data[x+'_2014'] > data[x+'_2012']) | (data['publication_date'].dt.year == 2013) & (data[x+'_2015'] > data[x+'_2013']) | (data['publication_date'].dt.year == 2014) & (data[x+'_2016'] > data[x+'_2014']) | (data['publication_date'].dt.year == 2015) & (data[x+'_2017'] > data[x+'_2015']) | (data['publication_date'].dt.year == 2016) & (data[x+'_2018'] > data[x+'_2016']) | (data['publication_date'].dt.year == 2017) & (data[x+'_2019'] > data[x+'_2017']),
    (data['publication_date'].dt.year == 2010) & (data[x+'_2012'] < data[x+'_2010']) | (data['publication_date'].dt.year == 2011) & (data[x+'_2013'] < data[x+'_2011']) | (data['publication_date'].dt.year == 2012) & (data[x+'_2014'] < data[x+'_2012']) | (data['publication_date'].dt.year == 2013) & (data[x+'_2015'] < data[x+'_2013']) | (data['publication_date'].dt.year == 2014) & (data[x+'_2016'] < data[x+'_2014']) | (data['publication_date'].dt.year == 2015) & (data[x+'_2017'] < data[x+'_2015']) | (data['publication_date'].dt.year == 2016) & (data[x+'_2018'] < data[x+'_2016']) | (data['publication_date'].dt.year == 2017) & (data[x+'_2019'] < data[x+'_2017'])
    ]
    values = ['0', '1', '-1']
    data[x+'_label_2'] = np.select(conditions, values, default=None)

In [ ]:
# Creating the label for the 3-year change 

for x in str_cols: 
    conditions = [ 
    (data['publication_date'].dt.year == 2010) & (data[x+'_2013'] == data[x+'_2010']) | (data['publication_date'].dt.year == 2011) & (data[x+'_2014'] == data[x+'_2011']) | (data['publication_date'].dt.year == 2012) & (data[x+'_2015'] == data[x+'_2012']) | (data['publication_date'].dt.year == 2013) & (data[x+'_2016'] == data[x+'_2013']) | (data['publication_date'].dt.year == 2014) & (data[x+'_2017'] == data[x+'_2014']) | (data['publication_date'].dt.year == 2015) & (data[x+'_2018'] == data[x+'_2015']) | (data['publication_date'].dt.year == 2016) & (data[x+'_2019'] == data[x+'_2016']),
    (data['publication_date'].dt.year == 2010) & (data[x+'_2013'] > data[x+'_2010']) | (data['publication_date'].dt.year == 2011) & (data[x+'_2014'] > data[x+'_2011']) | (data['publication_date'].dt.year == 2012) & (data[x+'_2015'] > data[x+'_2012']) | (data['publication_date'].dt.year == 2013) & (data[x+'_2016'] > data[x+'_2013']) | (data['publication_date'].dt.year == 2014) & (data[x+'_2017'] > data[x+'_2014']) | (data['publication_date'].dt.year == 2015) & (data[x+'_2018'] > data[x+'_2015']) | (data['publication_date'].dt.year == 2016) & (data[x+'_2019'] > data[x+'_2016']),
    (data['publication_date'].dt.year == 2010) & (data[x+'_2013'] < data[x+'_2010']) | (data['publication_date'].dt.year == 2011) & (data[x+'_2014'] < data[x+'_2011']) | (data['publication_date'].dt.year == 2012) & (data[x+'_2015'] < data[x+'_2012']) | (data['publication_date'].dt.year == 2013) & (data[x+'_2016'] < data[x+'_2013']) | (data['publication_date'].dt.year == 2014) & (data[x+'_2017'] < data[x+'_2014']) | (data['publication_date'].dt.year == 2015) & (data[x+'_2018'] < data[x+'_2015']) | (data['publication_date'].dt.year == 2016) & (data[x+'_2019'] < data[x+'_2016'])
    ]
    values = ['0', '1', '-1']
    data[x+'_label_3'] = np.select(conditions, values, default=None)

In [ ]:
# Creating the label for the 4-year change 

for x in str_cols: 
    conditions = [ 
    (data['publication_date'].dt.year == 2010) & (data[x+'_2014'] == data[x+'_2010']) | (data['publication_date'].dt.year == 2011) & (data[x+'_2015'] == data[x+'_2011']) | (data['publication_date'].dt.year == 2012) & (data[x+'_2016'] == data[x+'_2012']) | (data['publication_date'].dt.year == 2013) & (data[x+'_2017'] == data[x+'_2013']) | (data['publication_date'].dt.year == 2014) & (data[x+'_2018'] == data[x+'_2014']) | (data['publication_date'].dt.year == 2015) & (data[x+'_2019'] == data[x+'_2015']),
    (data['publication_date'].dt.year == 2010) & (data[x+'_2014'] > data[x+'_2010']) | (data['publication_date'].dt.year == 2011) & (data[x+'_2015'] > data[x+'_2011']) | (data['publication_date'].dt.year == 2012) & (data[x+'_2016'] > data[x+'_2012']) | (data['publication_date'].dt.year == 2013) & (data[x+'_2017'] > data[x+'_2013']) | (data['publication_date'].dt.year == 2014) & (data[x+'_2018'] > data[x+'_2014']) | (data['publication_date'].dt.year == 2015) & (data[x+'_2019'] > data[x+'_2015']),
    (data['publication_date'].dt.year == 2010) & (data[x+'_2014'] < data[x+'_2010']) | (data['publication_date'].dt.year == 2011) & (data[x+'_2015'] < data[x+'_2011']) | (data['publication_date'].dt.year == 2012) & (data[x+'_2016'] < data[x+'_2012']) | (data['publication_date'].dt.year == 2013) & (data[x+'_2017'] < data[x+'_2013']) | (data['publication_date'].dt.year == 2014) & (data[x+'_2018'] < data[x+'_2014']) | (data['publication_date'].dt.year == 2015) & (data[x+'_2019'] < data[x+'_2015'])
    ]
    values = ['0', '1', '-1']
    data[x+'_label_4'] = np.select(conditions, values, default=None)

In [ ]:
# Creating the label for the 5-year change 

for x in str_cols: 
    conditions = [ 
    (data['publication_date'].dt.year == 2010) & (data[x+'_2015'] == data[x+'_2010']) | (data['publication_date'].dt.year == 2011) & (data[x+'_2016'] == data[x+'_2011']) | (data['publication_date'].dt.year == 2012) & (data[x+'_2017'] == data[x+'_2012']) | (data['publication_date'].dt.year == 2013) & (data[x+'_2018'] == data[x+'_2013']) | (data['publication_date'].dt.year == 2014) & (data[x+'_2019'] == data[x+'_2014']),
    (data['publication_date'].dt.year == 2010) & (data[x+'_2015'] > data[x+'_2010']) | (data['publication_date'].dt.year == 2011) & (data[x+'_2016'] > data[x+'_2011']) | (data['publication_date'].dt.year == 2012) & (data[x+'_2017'] > data[x+'_2012']) | (data['publication_date'].dt.year == 2013) & (data[x+'_2018'] > data[x+'_2013']) | (data['publication_date'].dt.year == 2014) & (data[x+'_2019'] > data[x+'_2014']),
    (data['publication_date'].dt.year == 2010) & (data[x+'_2015'] < data[x+'_2010']) | (data['publication_date'].dt.year == 2011) & (data[x+'_2016'] < data[x+'_2011']) | (data['publication_date'].dt.year == 2012) & (data[x+'_2017'] < data[x+'_2012']) | (data['publication_date'].dt.year == 2013) & (data[x+'_2018'] < data[x+'_2013']) | (data['publication_date'].dt.year == 2014) & (data[x+'_2019'] < data[x+'_2014'])
    ]
    values = ['0', '1', '-1']
    data[x+'_label_5'] = np.select(conditions, values, default=None)

In [ ]:
# Creating the label for the 6-year change 

for x in str_cols: 
    conditions = [ 
    (data['publication_date'].dt.year == 2010) & (data[x+'_2016'] == data[x+'_2010']) | (data['publication_date'].dt.year == 2011) & (data[x+'_2017'] == data[x+'_2011']) | (data['publication_date'].dt.year == 2012) & (data[x+'_2018'] == data[x+'_2012']) | (data['publication_date'].dt.year == 2013) & (data[x+'_2019'] == data[x+'_2013']),
    (data['publication_date'].dt.year == 2010) & (data[x+'_2016'] > data[x+'_2010']) | (data['publication_date'].dt.year == 2011) & (data[x+'_2017'] > data[x+'_2011']) | (data['publication_date'].dt.year == 2012) & (data[x+'_2018'] > data[x+'_2012']) | (data['publication_date'].dt.year == 2013) & (data[x+'_2019'] > data[x+'_2013']),
    (data['publication_date'].dt.year == 2010) & (data[x+'_2016'] < data[x+'_2010']) | (data['publication_date'].dt.year == 2011) & (data[x+'_2017'] < data[x+'_2011']) | (data['publication_date'].dt.year == 2012) & (data[x+'_2018'] < data[x+'_2012']) | (data['publication_date'].dt.year == 2013) & (data[x+'_2019'] < data[x+'_2013'])
    ]
    values = ['0', '1', '-1']
    data[x+'_label_6'] = np.select(conditions, values, default=None)

In [ ]:
# Creating the label for the 7-year change 

for x in str_cols: 
    conditions = [ 
    (data['publication_date'].dt.year == 2010) & (data[x+'_2017'] == data[x+'_2010']) | (data['publication_date'].dt.year == 2011) & (data[x+'_2018'] == data[x+'_2011']) | (data['publication_date'].dt.year == 2012) & (data[x+'_2019'] == data[x+'_2012']),
    (data['publication_date'].dt.year == 2010) & (data[x+'_2017'] > data[x+'_2010']) | (data['publication_date'].dt.year == 2011) & (data[x+'_2018'] > data[x+'_2011']) | (data['publication_date'].dt.year == 2012) & (data[x+'_2019'] > data[x+'_2012']),
    (data['publication_date'].dt.year == 2010) & (data[x+'_2017'] < data[x+'_2010']) | (data['publication_date'].dt.year == 2011) & (data[x+'_2018'] < data[x+'_2011']) | (data['publication_date'].dt.year == 2012) & (data[x+'_2019'] < data[x+'_2012'])
    ]
    values = ['0', '1', '-1']
    data[x+'_label_7'] = np.select(conditions, values, default=None)

In [ ]:
# Creating the label for the 8-year change 

for x in str_cols: 
    conditions = [ 
    (data['publication_date'].dt.year == 2010) & (data[x+'_2018'] == data[x+'_2010']) | (data['publication_date'].dt.year == 2011) & (data[x+'_2019'] == data[x+'_2011']),
    (data['publication_date'].dt.year == 2010) & (data[x+'_2018'] > data[x+'_2010']) | (data['publication_date'].dt.year == 2011) & (data[x+'_2019'] > data[x+'_2011']),
    (data['publication_date'].dt.year == 2010) & (data[x+'_2018'] < data[x+'_2010']) | (data['publication_date'].dt.year == 2011) & (data[x+'_2019'] < data[x+'_2011'])
    ]
    values = ['0', '1', '-1']
    data[x+'_label_8'] = np.select(conditions, values, default=None)

In [ ]:
# Creating the label for the 9-year change 

for x in str_cols: 
    conditions = [ 
    (data['publication_date'].dt.year == 2010) & (data[x+'_2019'] == data[x+'_2010']),
    (data['publication_date'].dt.year == 2010) & (data[x+'_2019'] > data[x+'_2010']),
    (data['publication_date'].dt.year == 2010) & (data[x+'_2019'] < data[x+'_2010'])
    ]
    values = ['0', '1', '-1']
    data[x+'_label_9'] = np.select(conditions, values, default=None)

In [ ]:
# Save the final labeled dataset. It is stored in the two zipped data files on this directory.
data.to_csv('data.csv', index=False) 